In [ ]:
!pip install markdown
import markdown


In [ ]:
!pip install -U sentence-transformers rank_bm25

In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from sentence_transformers.evaluation import TripletEvaluator
import random
from collections import defaultdict
import os
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback
from sentence_transformers.losses import TripletDistanceMetric
from sklearn.model_selection import train_test_split


In [ ]:
from torch.utils.data import DataLoader
import math
import torch
import random
import numpy as np
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv

In [ ]:
pd.set_option('mode.chained_assignment', None)


## Load python lib

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)



In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import csv
import io
import concurrent.futures
from tqdm.auto import tqdm
import os
import re
import torch

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
import torch

deps = ["datasets" ,"sentence-transformers", "rich['jupyter']", "requests"]
if torch.cuda.is_available():
    deps.append("faiss-gpu")
else:
    deps.append("faise-cpu")

!pip install huggingface_hub
!pip install rich
! pip install pandas_profiling
!pip install {" ".join(deps)} --upgrade

from huggingface_hub import hf_api
import requests
import pandas as pd
import matplotlib.pyplot as plt
import rich
import json
import re
import ast
from collections import Counter
%matplotlib inline
plt.style.use("ggplot")
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)
import datetime as dt
from collections import defaultdict



In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")



## Load Data

In [ ]:
import pandas as pd

def string_to_list( input_string):
    if type(input_string) != str: return[]
    if input_string[0] != '[': return [input_string]
    return ast.literal_eval(input_string)

data_full = pd.read_csv("/content/gdrive/MyDrive/final project results/model_EDA_final_2.csv")
columns_to_keep = ['modelId', 'downloads','readme']
data_full = data_full[columns_to_keep]

In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/final project results/model_preprocessed_by_Chaim.csv")
data = data.drop(columns=['col_to_embedd'])

In [ ]:
# Perform a left join on 'id' and 'modelId'
data = data.merge(data_full, left_on='id', right_on='modelId', how='left')
data['downloads'] = data['downloads'].fillna(0)  # Fill missing values with 0 if needed
data['col_to_embedd']  = data["readme"]
data = data.drop(columns=['modelId'])

In [ ]:
# data_full = pd.read_csv("/content/gdrive/MyDrive/final project results/model_EDA_final.csv")
# data_full.sample(n=10)

In [ ]:
data.columns

Index(['Unnamed: 0', 'index', 'id', 'labels_col', 'datasets', 'pipeline_tag',
       'downloads', 'readme', 'col_to_embedd'],
      dtype='object')

In [ ]:
# # Sort by "downloads" in descending order
# data.sort_values(by="downloads", ascending=False, inplace=True)
# data.drop_duplicates(subset="col_to_embedd", keep="first", inplace=True)
# data.reset_index(drop=True, inplace=True)


### preprocess for basic modeling

In [ ]:
data["id"] = data["id"].astype(str)
data["col_to_embedd"] = data["col_to_embedd"].astype(str)
data['paragraphs']= data["col_to_embedd"]

data["datasets"] = data["datasets"].apply(string_to_list)
data["labels_col"] = data["labels_col"].apply(string_to_list)

passages = data['paragraphs'].tolist()
model_id_lst = data['id'].tolist()

## Make Triplets

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sentence_transformers import InputExample
import random
class Create_triplets:

    def __init__(self, model_df, datasets_df) -> None:

        self.model_df = model_df
        self.datasets_df = datasets_df
        self.triplets = []

    def generate_single_triplet(self, row, relevant_dataset):
        # split positive and negative
        cur_task_dataset = relevant_dataset[relevant_dataset["pipeline_tag"] == row["pipeline_tag"]]
        not_task_dataset = relevant_dataset[relevant_dataset["pipeline_tag"] != row["pipeline_tag"]]
        # init loop
        count =0
        anchor_sentence = ""
        positive_sentence =""
        single_triplet = ""
        while (count < 5) and (anchor_sentence == positive_sentence):
            # top percentile of in-task model
            percentile_75 = cur_task_dataset['intersection_size'].quantile(0.75)
            max_intesection_group =cur_task_dataset[(cur_task_dataset['id'] != row["id"]) & (cur_task_dataset['intersection_size'] >= percentile_75) ]
            if max_intesection_group.shape[0] > 1:
              max_intersection_row = max_intesection_group.sample(n=1).iloc[0]
            else:
              break
            # bottom percentile of out of task
            percentile_25 = not_task_dataset['intersection_size'].quantile(0.25)
            min_intersection_row = not_task_dataset[not_task_dataset['intersection_size'] <= percentile_25 ].sample(n=1).iloc[0]

            anchor_sentence = row["col_to_embedd"]
            positive_sentence = max_intersection_row["col_to_embedd"]
            negative_sentence = min_intersection_row["col_to_embedd"]
            single_triplet = InputExample(texts=[anchor_sentence, positive_sentence, negative_sentence])
            count+=1
        return single_triplet


    def generate_hard_single_triplet(self, row, relevant_dataset):
        # split positive and negative
        cur_task_dataset = relevant_dataset[relevant_dataset["pipeline_tag"] == row["pipeline_tag"]]
        cur_task_dataset = relevant_dataset[relevant_dataset["pipeline_tag"] == row["pipeline_tag"]]
        # init loop
        count =0
        anchor_sentence = ""
        positive_sentence =""
        single_triplet = ""
        while (count < 5) and (anchor_sentence == positive_sentence):
            # top percentile of in-task model
            percentile_75 = cur_task_dataset['intersection_size'].quantile(0.75)
            max_intesection_group =cur_task_dataset[(cur_task_dataset['id'] != row["id"]) & (cur_task_dataset['intersection_size'] >= percentile_75) ]
            if max_intesection_group.shape[0] > 1:
              max_intersection_row = max_intesection_group.sample(n=1).iloc[0]
            else:
              break
            # bottom percentile of out of task
            percentile_10 = cur_task_dataset['intersection_size'].quantile(0.10)
            min_intersection_row = cur_task_dataset[cur_task_dataset['intersection_size'] <= percentile_10 ].sample(n=1).iloc[0]

            anchor_sentence = row["col_to_embedd"]
            positive_sentence = max_intersection_row["col_to_embedd"]
            negative_sentence = min_intersection_row["col_to_embedd"]
            single_triplet = InputExample(texts=[anchor_sentence, positive_sentence, negative_sentence])
            count+=1
        return single_triplet



    def smart_triplets_model_to_model(self):
        ## Filter on rows minimal length
        relevant_dataset = self.model_df[self.model_df["col_to_embedd"].str.len() > 300]
        for index, row in relevant_dataset.sample(n=20000 ,replace = True).iterrows():
            initial_labels = set(row["labels_col"])
            if len(initial_labels) > 3  and  row["downloads"]> 5:
                # weighted score
                relevant_dataset.loc[:, "intersection_size"] = relevant_dataset['labels_col'].apply(lambda x: len(set(x) & initial_labels))
                # relevant_dataset["intersection_size"] = relevant_dataset['labels_col'].apply(lambda x: len(set(x) & initial_labels))

                # triplets
                single_triplet = self.generate_single_triplet(row, relevant_dataset)
                if single_triplet != "":
                  self.triplets.append(single_triplet)
        return self.triplets

    def weighted_smart_triplets_model_to_model(self):
        wighted_shared_labels_dict = self.create_weighted_label_model_dict(self.model_df, "id", "labels_col")
        relevant_dataset = self.model_df[self.model_df["col_to_embedd"].str.len() > 300]
        length = relevant_dataset.shape[0]
        for i, (index, row) in enumerate(relevant_dataset.sample(n=20000 ,replace = True).iterrows()):
            print(f"creating triplet {i} / {length}", end='\r')
            initial_labels = set(row["labels_col"])
            if len(initial_labels) > 3 and  row["downloads"]> 5:
                # weighted score
                relevant_dataset['intersection_labels'] = relevant_dataset['labels_col'].apply(lambda x: set(x) & initial_labels)
                relevant_dataset.loc[:, "intersection_size"]  = relevant_dataset['intersection_labels'].apply(lambda row: self.calc_weighted_score(row, wighted_shared_labels_dict))

                # triplets
                single_triplet = self.generate_single_triplet(row, relevant_dataset)
                if single_triplet != None: self.triplets.append(single_triplet)
        return self.triplets

    def weighted_hard_triplets_model_to_model(self):
        wighted_shared_labels_dict = self.create_weighted_label_model_dict(self.model_df, "id", "labels_col")
        relevant_dataset = self.model_df[self.model_df["col_to_embedd"].str.len() > 300]
        length = relevant_dataset.shape[0]
        for i, (index, row) in enumerate(relevant_dataset.sample(n=20000 ,replace = True).iterrows()):
            print(f"creating triplet {i} / {length}", end='\r')
            initial_labels = set(row["labels_col"])
            if len(initial_labels) > 3 and  row["downloads"]> 5:
                # weighted score
                relevant_dataset['intersection_labels'] = relevant_dataset['labels_col'].apply(lambda x: set(x) & initial_labels)
                relevant_dataset.loc[:, "intersection_size"]  = relevant_dataset['intersection_labels'].apply(lambda row: self.calc_weighted_score(row, wighted_shared_labels_dict))

                # triplets
                single_triplet = self.generate_hard_single_triplet(row, relevant_dataset)
                if single_triplet != None: self.triplets.append(single_triplet)
        return self.triplets



    def create_weighted_label_model_dict(self, df, model_id_column, label_column):
            label_model_dict = defaultdict(set)
            for index, row in df.iterrows():
                model_id = row[model_id_column]
                labels = (row[label_column])
                if not labels: continue
                for label in labels:
                    label_model_dict[label].add(model_id)

            label_model_dict = calculate_quantiles_and_convert_to_score(label_model_dict)
            return label_model_dict


    def calc_weighted_score(self, row , wighted_shared_labels_dict):
            wighted_score = 0
            for label in row:
                wighted_score += wighted_shared_labels_dict[label]
            return wighted_score


    def create_triplets(self):
        self.weighted_smart_triplets_model_to_model()
        # self.triplets_dataset_to_model()
        # self.triplets_dataset_to_dataset()
        # self.triplets_model_to_model()

        return self.triplets


def calculate_quantiles_and_convert_to_score(count_dict):
    values = [len(labels) for labels in count_dict.values()]
    quantiles = np.percentile(values, [20, 40, 60, 80])
    score_dict = {}
    for key, value in count_dict.items():
        if len(value) <= quantiles[0]:
            score = 2.5
        elif len(value) <= quantiles[1]:
            score = 2.5
        elif len(value) <= quantiles[2]:
            score = 2
        elif len(value) <= quantiles[3]:
            score = 1.5
        else:
            score = 1
        score_dict[key] = score
    return score_dict


## Train Class

In [ ]:
class SentenceEvaluator:
    """
    Base class for all evaluators

    Extend this class and implement __call__ for custom evaluators.
    """

    def __call__(self, model, output_path: str = None, epoch: int = -1, steps: int = -1) -> float:
        """
        This is called during training to evaluate the model.
        It returns a score for the evaluation with a higher score indicating a better result.
t
        :param model:
            the model to evaluate
        :param output_path:
            path where predictions and metrics are written to
        :param epoch
            the epoch where the evaluation takes place.
            This is used for the file prefixes.
            If this is -1, then we assume evaluation on test data.
        :param steps
            the steps in the current epoch at time of the evaluation.
            This is used for the file prefixes.
            If this is -1, then we assume evaluation at the end of the epoch.
        :return: a score for the evaluation with a higher score indicating a better result
        """
        pass

class PercentRecommendations(SentenceEvaluator):
  def __init__(self, data):
    self.data = data
    self.passages = None
    self.model_id_lst = None

  def prepare_data_for_eval (self, data):
    data["id"] = data["id"].astype(str)
    data["col_to_embedd"] = data["col_to_embedd"].astype(str)
    data['paragraphs']= data["col_to_embedd"]

    data["datasets"] = data["datasets"].apply(string_to_list)
    data["labels_col"] = data["labels_col"].apply(string_to_list)

    passages = data['paragraphs'].tolist()
    model_id_lst = data['id'].tolist()
    return data ,passages, model_id_lst

  def __call__(self, model, output_path , epoch , steps ):
    """
    :param model:
        the model to evaluate
    :param output_path:
        path where predictions and metrics are written to
    :param epoch
        the epoch where the evaluation takes place.
        This is used for the file prefixes.
        If this is -1, then we assume evaluation on test data.
    :param steps
        the steps in the current epoch at time of the evaluation.
        This is used for the file prefixes.
        If this is -1, then we assume evaluation at the end of the epoch.
    :return: a score for the evaluation with a higher score indicating a better result
    """
    data ,passages  , model_id_lst = self.prepare_data_for_eval(self.data)
    corpus_embeddings = model.encode(passages, convert_to_tensor=True, show_progress_bar=True)
    retrieved_reccomendations = self.retrieve_reccomendations_same_objects(model, corpus_embeddings ,corpus_embeddings, data, 10)
    res1 = self.evaluate_by_intersection_size(retrieved_reccomendations , self.percentage_label_intersections )
    print(res1)

  def retrieve_reccomendations_same_objects(self,model, ds_query_embeddings ,ds_target_embeddings, df_query, k):
      recommendations_and_ground_truth = dict()
      raw_recommendations = util.semantic_search(torch.Tensor(ds_query_embeddings), torch.Tensor(ds_target_embeddings))
      for index, recommendations_list in enumerate(raw_recommendations):
          print(f"retriveing recommendations for index {index}/{ds_query_embeddings.shape[0]}", end='\r')
          query_id = df_query.iloc[index]['id']
          recommendations_id = [df_query.iloc[recommendation['corpus_id']]['id'] for recommendation in recommendations_list]
          recommendations_labels = [df_query.iloc[recommendation['corpus_id']]['labels_col'] for recommendation in recommendations_list]
          ground_truth = df_query[df_query["id"]==query_id]["labels_col"].iloc[0]
          recommendations_and_ground_truth[query_id] = (recommendations_id, recommendations_labels, ground_truth)
      return recommendations_and_ground_truth


  def percentage_label_intersections(self, label_set_recommended, label_set_grount_truth):
    counts = np.zeros(len(label_set_recommended))
    for i, label_set in enumerate(label_set_recommended):
      intersection_set = set(label_set) & set(label_set_grount_truth)
      counts[i] = len(intersection_set) / min(len(label_set), len(label_set_grount_truth))
    if len(counts) == 0:
      return 0,0,0
    return counts.max(), counts.mean(), counts.min()

  def count_label_intersections(self, label_set_recommended, label_set_grount_truth):
    counts = np.zeros(len(label_set_recommended))
    for i, label_set in enumerate(label_set_recommended):

        intersection_set = set(label_set) & set(label_set_grount_truth)
        counts[i] = len(intersection_set)
    return counts.max(), counts.mean(), counts.min()

  def evaluate_by_intersection_size(self, retrieveed_reccomendations: dict, func_label_intersection) -> pd.DataFrame:
    ks = [1, 3, 5, 10]
    res_intersection_max = np.zeros((len(retrieveed_reccomendations), len(ks)))
    res_intersection_mean = np.zeros((len(retrieveed_reccomendations), len(ks)))
    res_intersection_min = np.zeros((len(retrieveed_reccomendations), len(ks)))
    for i, (id, (recomendation_id, recommendations_labels, ground_truth)) in enumerate(retrieveed_reccomendations.items()):
        for j, k in enumerate(ks):
            k = min(len(ground_truth), k, len(recommendations_labels))
            intersection_max, intersection_mean, intersection_min = func_label_intersection(recommendations_labels[:k], ground_truth[:k])
            res_intersection_max[i, j] = intersection_max
            res_intersection_mean[i, j] = intersection_mean
            res_intersection_min[i, j] = intersection_min

    data = [res_intersection_max.mean(axis=0), res_intersection_mean.mean(axis=0), res_intersection_min.mean(axis=0)]
    index = ['intersection_max', 'intersection_mean', 'intersection_min']
    columns = [f"k={k}" for k in ks]
    res = pd.DataFrame(data=data, columns=columns, index=index)
    return res

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback
from sentence_transformers.losses import TripletDistanceMetric
from sklearn.model_selection import train_test_split
class Train_triplets:

    def __init__(self, model_name, models_df, datasets_df, output_path):
        word_embedding_model = models.Transformer(model_name)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                      pooling_mode_mean_tokens=True,
                                      pooling_mode_cls_token=False,
                                      pooling_mode_max_tokens=False)
        model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

        self.device = device #torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.models_df = models_df
        self.datasets_df = datasets_df
        self.triplets_creator = Create_triplets(self.models_df, self.datasets_df)
        self.output_path = output_path


    def train(self , input_triplets, output_model_name):
        num_epochs = 5
        tb_writer = SummaryWriter(log_dir=os.path.join(self.output_path, "logs"))
        tb_callback = TensorBoardCallback(tb_writer=tb_writer)

         # Create a DataLoader with your training triplets
        triplets = input_triplets
        triplets_train, triplets_test = train_test_split(triplets)
        train_dataset = SentencesDataset(examples=triplets_train, model=self.model)
        train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8,
                                    pin_memory=True, num_workers=4)
        warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)

        steps_per_epoch_num = 200
        # Define the triplet loss function
        train_loss = losses.TripletLoss(model=self.model, distance_metric=TripletDistanceMetric.COSINE)

        # Define a TripletEvaluator for evaluation during fine-tuning
        triplet_evaluator = TripletEvaluator.from_input_examples(examples=triplets_test)
        #precent_of_reccomendations = PercentRecommendations(data)

        # Fine-tune the model with mixed precision
        self.model.fit(train_objectives=[(train_dataloader, train_loss)],
                       evaluator= triplet_evaluator,
                       epochs=num_epochs,
                       steps_per_epoch = steps_per_epoch_num,
                       warmup_steps=warmup_steps,
                       evaluation_steps=2,
                       use_amp=True,
                    #    callback=tb_callback.tb_writer,
                       checkpoint_path= os.path.join(self.output_path, output_model_name, "checkpoints"),
                       checkpoint_save_steps = 250,
                       output_path = self.output_path)
        return self.model


## Load Data

In [ ]:
import pandas as pd

def string_to_list( input_string):
    if type(input_string) != str: return[]
    if input_string[0] != '[': return [input_string]
    return ast.literal_eval(input_string)

data_full = pd.read_csv("/content/gdrive/MyDrive/final project results/model_EDA_final_2.csv")
columns_to_keep = ['modelId', 'downloads','readme']
data_full = data_full[columns_to_keep]

In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/final project results/model_preprocessed_by_Chaim.csv")
data = data.drop(columns=['col_to_embedd'])


In [ ]:
data.columns

Index(['Unnamed: 0', 'index', 'id', 'labels_col', 'datasets', 'pipeline_tag'], dtype='object')

In [ ]:
# Perform a left join on 'id' and 'modelId'
data = data.merge(data_full, left_on='id', right_on='modelId', how='left')
data['downloads'] = data['downloads'].fillna(0)  # Fill missing values with 0 if needed
data['col_to_embedd']  = data["readme"]
data = data.drop(columns=['modelId'])

In [ ]:
def remove_substrings(input_string, substrings):
    for substring in substrings:
        input_string = input_string.replace(substring, "")
    return input_string


def capture_name_basis(input_str):
  replace_size_params = ["-base",'-tiny',"-medium","-small","-large","-xl","-xs","xlm","-v0",'-v1',"-v2","v3","-v4","-updated","-finetuned","-uncased" ,"7b","13b"]
  replace_size_params_underscore = ["_base",'_tiny',"_medium","_small","_large","_xl","_xs","_xlm","_v0",'-v1',"-v2","_v3","_v4","-updated","_finetuned","_uncased"]
  result = remove_substrings(input_str, replace_size_params)
  result = remove_substrings(input_str, replace_size_params_underscore)

  parts = input_str.split("-")
  if len(parts) >= 3:
      result = "-".join(parts[0:3])
  else:
      result = input_str[:-3]
  return result[:-2]

data["name_basis"] = data["id"].apply(capture_name_basis)


In [ ]:
data.columns

Index(['Unnamed: 0', 'index', 'id', 'labels_col', 'datasets', 'pipeline_tag',
       'downloads', 'readme', 'col_to_embedd', 'name_basis'],
      dtype='object')

### Remove duplicates (same name root or same readme)

In [ ]:
# Deduplicate
data.sort_values(by="downloads", ascending=False, inplace=True)

print(data.shape[0])
data.drop_duplicates(subset="col_to_embedd", keep="first", inplace=True)

print(data.shape[0])
data.drop_duplicates(subset="name_basis", keep="first", inplace=True)
print(data.shape[0])

data.reset_index(drop=True, inplace=True)


52843
39820
31178


In [ ]:
textual_queries = dict()
for name_basis in lst_model:
    model_id = data[data["name_basis"] == name_basis]["id"].iloc[0]
    textual_queries[model_id] = ""
textual_queries

NameError: ignored

### preprocess for basic modeling

In [ ]:
data["id"] = data["id"].astype(str)
data["col_to_embedd"] = data["col_to_embedd"].astype(str)
data['paragraphs']= data["col_to_embedd"]

data["datasets"] = data["datasets"].apply(string_to_list)
data["labels_col"] = data["labels_col"].apply(string_to_list)

passages = data['paragraphs'].tolist()
model_id_lst = data['id'].tolist()

## train triplets


In [ ]:
trainer1 = Train_triplets('thenlper/gte-small', data, data, "/content/gdrive/MyDrive/final project results/friday_gte_small1")
smart_triplets = trainer1.triplets_creator.smart_triplets_model_to_model()
trainer1.train(smart_triplets ,  "triplets")

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/100 [00:00<?, ?it/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
trainer2 = Train_triplets('thenlper/gte-small', data, data, "/content/gdrive/MyDrive/final project results/friday_gte_small2")
weighted_smart_triplets  = trainer2.triplets_creator.weighted_smart_triplets_model_to_model() #"weighted_triplets_full_data")
trainer2.train(weighted_smart_triplets , "weighted_smart_triplets")

<ipython-input-266-f3e3671e2211>:99: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  relevant_dataset.loc[:, "intersection_size"]  = relevant_dataset['intersection_labels'].apply(lambda row: self.calc_weighted_score(row, wighted_shared_labels_dict))


<ipython-input-266-f3e3671e2211>:99: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  relevant_dataset.loc[:, "intersection_size"]  = relevant_dataset['intersection_labels'].apply(lambda row: self.calc_weighted_score(row, wighted_shared_labels_dict))


<ipython-input-266-f3e3671e2211>:99: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  relevant_dataset.loc[:, "intersection_size"]  = relevant_dataset['intersection_labels'].apply(lambda row: self.calc_weighted_score(row, wighted_shared_labels_dict))
<ipython-input-266-f3e3671e2211>:99: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  relevant_dataset.loc[:, "intersection_size"]  = relevant_dataset['intersection_labels'].apply(lambda row: self.calc_weighted_score(row, wighted_shared_labels_dict))


<ipython-input-266-f3e3671e2211>:99: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  relevant_dataset.loc[:, "intersection_size"]  = relevant_dataset['intersection_labels'].apply(lambda row: self.calc_weighted_score(row, wighted_shared_labels_dict))
<ipython-input-266-f3e3671e2211>:99: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  relevant_dataset.loc[:, "intersection_size"]  = relevant_dataset['intersection_labels'].apply(lambda row: self.calc_weighted_score(row, wighted_shared_labels_dict))
<ipython-input-266-f3e3671e2211>:99: DeprecationWa

<ipython-input-266-f3e3671e2211>:99: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  relevant_dataset.loc[:, "intersection_size"]  = relevant_dataset['intersection_labels'].apply(lambda row: self.calc_weighted_score(row, wighted_shared_labels_dict))


...

Iteration:   0%|          | 0/200 [00:00<?, ?it/s]

Iteration:   0%|          | 0/200 [00:00<?, ?it/s]

Iteration:   0%|          | 0/200 [00:00<?, ?it/s]

Iteration:   0%|          | 0/200 [00:00<?, ?it/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
trainer3 = Train_triplets('thenlper/gte-small', data, data, "/content/gdrive/MyDrive/final project results/friday_gte_small")
weighted_hard_triplets  = trainer3.triplets_creator.weighted_hard_triplets_model_to_model() #"hard_weighted_triplets_full_data")
trainer3.train(weighted_hard_triplets, "weighted_hard_triplets")


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: ignored